In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
from sqlalchemy import create_engine
# Postgres username, password, and database name
POSTGRES_ADDRESS = 'lhl-data-bootcamp.crzjul5qln0e.ca-central-1.rds.amazonaws.com' ## INSERT YOUR DB ADDRESS IF IT'S NOT ON PANOPLY
POSTGRES_PORT = '5432'
POSTGRES_USERNAME = 'lhl_student' ## CHANGE THIS TO YOUR PANOPLY/POSTGRES USERNAME
POSTGRES_PASSWORD = 'lhl_student' ## CHANGE THIS TO YOUR PANOPLY/POSTGRES PASSWORD
POSTGRES_DBNAME = 'mid_term_project' ## CHANGE THIS TO YOUR DATABASE NAME
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(username=POSTGRES_USERNAME,password=POSTGRES_PASSWORD,ipaddress=POSTGRES_ADDRESS,port=POSTGRES_PORT,dbname=POSTGRES_DBNAME))
# Create the connection
cnx = create_engine(postgres_str)

In [3]:
X = pd.read_csv("DATAFRAMETOTEST.csv")

In [4]:
loaded_model = pickle.load(open("Saved Models/ModelRanForQuart.sav",'rb'))

In [5]:
y = loaded_model.predict(X)

In [6]:
len(y)

143946

In [7]:
pd.value_counts(y)

3    25679
4    21067
2    18009
5    14430
1    13315
8    11893
6    11669
9    10654
0     8780
7     8450
dtype: int64

In [8]:
ywQ = []
for i in range(len(y)):
    if y[i] == 0:
        ywQ.append(-9.748352)
    if y[i] == 1:
        ywQ.append(-5.351785)
    if y[i] == 2:
        ywQ.append(-2.274553)
    if y[i] == 3:
        ywQ.append(0.477148)
    if y[i] == 4:
        ywQ.append(3.196825)
    if y[i] == 5:
        ywQ.append(6.26977)
    if y[i] == 6:
        ywQ.append(10)
    if y[i] == 7:
        ywQ.append(15.07837)
    if y[i] == 8:
        ywQ.append(24.251818)
    if y[i] == 9:
        ywQ.append("30+")

In [9]:
len(ywQ)

143946

In [10]:
X

,Month_1,Month_2,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,Month_10,...,dest_airport_id_16133,dest_airport_id_16218,dest_airport_id_16869,departures_scheduled_scaled,departures_performed_scaled,seats_scaled,passengers_scaled,freight_scaled,mail_scaled,distance_scaled
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0.090699,0.087961,0.035132,0.035698,0,0.000000,0.073910
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0.099316,0.097685,0.118200,0.140116,0,0.011052,0.386914
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0.137832,0.133492,0.164326,0.192277,0,0.000000,0.193659
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0.007303,0.007019,0.003825,0.003825,0,0.000000,0.085218
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0.003505,0.001755,0.000978,0.000735,0,0.000000,0.114297
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143941,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0.053657,0.053185,0.063276,0.057961,0,0.000000,0.134289
143942,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0.122334,0.120381,0.150435,0.175001,0,0.001859,0.342286
143943,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0.194542,0.191151,0.226524,0.206435,0,0.000000,0.055129
143944,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0.044984,0.043286,0.081453,0.084991,0,0.185924,0.995759


In [11]:
FLITEST = pd.read_sql_query('''DROP TABLE IF EXISTS passengers_aggregated;
DROP TABLE IF EXISTS week_one;

SELECT op_carrier_fl_num, op_unique_carrier, MAX(origin_airport_id) AS origin_airport_id, MAX(dest_airport_id) AS dest_airport_id, AVG(distance) AS distance, CONCAT(flights_test.origin_airport_id, '-', flights_test.dest_airport_id) AS route, EXTRACT(DAY from fl_date) AS day
INTO TABLE week_one
FROM flights_test
WHERE EXTRACT(DAY from fl_date) < 08
GROUP BY op_carrier_fl_num, route, op_unique_carrier, day;

SELECT AVG(departures_scheduled) AS avg_departures_scheduled, AVG(departures_performed) AS avg_departures_performed, AVG(seats) AS avg_seats, AVG(passengers) AS avg_passengers, AVG(freight) AS avg_freight, AVG(mail) AS avg_mail, MAX(aircraft_group) AS aircraft_group, MAX(aircraft_type) AS aircraft_type, unique_carrier, CONCAT(origin_airport_id, '-', dest_airport_id) AS route
INTO passengers_aggregated
FROM passengers
WHERE month = 01
GROUP BY route, unique_carrier;

SELECT day, op_unique_carrier, MAX(week_one.origin_airport_id) AS origin_airport_id, MAX(week_one.dest_airport_id) AS dest_airport_id, MAX(distance) AS distance, MAX(avg_departures_scheduled) AS avg_departures_scheduled, MAX(avg_departures_performed) AS avg_departures_performed, MAX(avg_seats) AS avg_seats, MAX(avg_passengers) AS avg_passengers, MAX(avg_freight), MAX(avg_mail) AS avg_mail, MAX(aircraft_group) AS aircraft_group, MAX(aircraft_type) AS aircraft_type
FROM week_one JOIN passengers_aggregated ON (week_one.route = passengers_aggregated.route AND op_unique_carrier = unique_carrier)
GROUP BY op_carrier_fl_num, week_one.route, op_unique_carrier, day;''',cnx)

In [12]:
FLITEST

,day,op_unique_carrier,origin_airport_id,dest_airport_id,distance,avg_departures_scheduled,avg_departures_performed,avg_seats,avg_passengers,max,avg_mail,aircraft_group,aircraft_type
0,4.0,OO,11337,13930,397.0,51.750000,50.125000,2513.750000,1894.625000,0.000000,0.000000,6,631
1,1.0,AA,10397,12892,1947.0,56.666667,55.666667,8457.500000,7436.500000,2044.833333,3934.000000,6,699
2,1.0,WN,13204,13232,990.0,78.642857,76.071429,11757.928571,10204.928571,14073.571429,0.000000,6,838
3,5.0,OO,10599,11298,597.0,2.000000,1.000000,70.000000,39.000000,0.000000,0.000000,6,631
4,3.0,9E,11433,14100,453.0,4.166667,4.000000,273.666667,203.000000,0.000000,0.000000,6,638
...,...,...,...,...,...,...,...,...,...,...,...,...,...
143941,2.0,OO,13264,12892,630.0,32.750000,32.250000,2113.000000,1402.000000,0.000000,0.000000,6,673
143942,4.0,AA,14908,13930,1726.0,69.800000,68.600000,10764.000000,9288.000000,32.800000,661.800000,6,614
143943,4.0,WN,14107,14679,304.0,111.000000,108.928571,16208.357143,10956.357143,8370.928571,0.000000,6,838
143944,5.0,UA,11618,12173,4962.0,25.666667,24.666667,5828.166667,4510.833333,107314.166667,66181.666667,6,626


In [13]:
FLITEST["predicted dely"] = ywQ

In [14]:
FLITEST

,day,op_unique_carrier,origin_airport_id,dest_airport_id,distance,avg_departures_scheduled,avg_departures_performed,avg_seats,avg_passengers,max,avg_mail,aircraft_group,aircraft_type,predicted dely
0,4.0,OO,11337,13930,397.0,51.750000,50.125000,2513.750000,1894.625000,0.000000,0.000000,6,631,30+
1,1.0,AA,10397,12892,1947.0,56.666667,55.666667,8457.500000,7436.500000,2044.833333,3934.000000,6,699,3.196825
2,1.0,WN,13204,13232,990.0,78.642857,76.071429,11757.928571,10204.928571,14073.571429,0.000000,6,838,10
3,5.0,OO,10599,11298,597.0,2.000000,1.000000,70.000000,39.000000,0.000000,0.000000,6,631,-9.748352
4,3.0,9E,11433,14100,453.0,4.166667,4.000000,273.666667,203.000000,0.000000,0.000000,6,638,-9.748352
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143941,2.0,OO,13264,12892,630.0,32.750000,32.250000,2113.000000,1402.000000,0.000000,0.000000,6,673,6.26977
143942,4.0,AA,14908,13930,1726.0,69.800000,68.600000,10764.000000,9288.000000,32.800000,661.800000,6,614,0.477148
143943,4.0,WN,14107,14679,304.0,111.000000,108.928571,16208.357143,10956.357143,8370.928571,0.000000,6,838,3.196825
143944,5.0,UA,11618,12173,4962.0,25.666667,24.666667,5828.166667,4510.833333,107314.166667,66181.666667,6,626,10


In [15]:
date = []
for i in range(len(y)):
    if FLITEST["day"][i] == 1.0:
        date.append("01-01-2020")
    if FLITEST["day"][i] == 2.0:
        date.append("01-02-2020")
    if FLITEST["day"][i] == 3.0:
        date.append("01-03-2020")
    if FLITEST["day"][i] == 4.0:
        date.append("01-04-2020")
    if FLITEST["day"][i] == 5.0:
        date.append("01-05-2020")
    if FLITEST["day"][i] == 6.0:
        date.append("01-06-2020")
    if FLITEST["day"][i] == 7.0:
        date.append("01-07-2020")

In [16]:
len(date)

143946

In [17]:
FLITEST["day"] = date

In [18]:
FLITEST.head()

,day,op_unique_carrier,origin_airport_id,dest_airport_id,distance,avg_departures_scheduled,avg_departures_performed,avg_seats,avg_passengers,max,avg_mail,aircraft_group,aircraft_type,predicted dely
0,01-04-2020,OO,11337,13930,397.0,51.750000,50.125000,2513.750000,1894.625000,0.000000,0.0,6,631,30+
1,01-01-2020,AA,10397,12892,1947.0,56.666667,55.666667,8457.500000,7436.500000,2044.833333,3934.0,6,699,3.196825
2,01-01-2020,WN,13204,13232,990.0,78.642857,76.071429,11757.928571,10204.928571,14073.571429,0.0,6,838,10
3,01-05-2020,OO,10599,11298,597.0,2.000000,1.000000,70.000000,39.000000,0.000000,0.0,6,631,-9.748352
4,01-03-2020,9E,11433,14100,453.0,4.166667,4.000000,273.666667,203.000000,0.000000,0.0,6,638,-9.748352


In [19]:
FLITEST.drop(columns=["avg_departures_scheduled", "avg_departures_performed","avg_seats", "avg_passengers","max","avg_mail","aircraft_group"], inplace = True)

In [20]:
FLITEST

,day,op_unique_carrier,origin_airport_id,dest_airport_id,distance,aircraft_type,predicted dely
0,01-04-2020,OO,11337,13930,397.0,631,30+
1,01-01-2020,AA,10397,12892,1947.0,699,3.196825
2,01-01-2020,WN,13204,13232,990.0,838,10
3,01-05-2020,OO,10599,11298,597.0,631,-9.748352
4,01-03-2020,9E,11433,14100,453.0,638,-9.748352
...,...,...,...,...,...,...,...
143941,01-02-2020,OO,13264,12892,630.0,673,6.26977
143942,01-04-2020,AA,14908,13930,1726.0,614,0.477148
143943,01-04-2020,WN,14107,14679,304.0,838,3.196825
143944,01-05-2020,UA,11618,12173,4962.0,626,10


In [21]:
FLITEST["predicted dely"].value_counts()

0.477148     25679
3.196825     21067
-2.274553    18009
6.26977      14430
-5.351785    13315
24.251818    11893
10           11669
30+          10654
-9.748352     8780
15.07837      8450
Name: predicted dely, dtype: int64

In [22]:
FLITEST.to_csv("Submission.csv",index = False)